In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
df.head()

In [ ]:
df.shape

In [ ]:
import tensorflow as tf
from tensorflow import keras

## Seperating Independent and dependent features

In [ ]:
x_train = df.drop(["label"],axis=1)
y_train = df["label"]

In [ ]:
print(type(y_train))

# Normalizing the data

In [ ]:
x_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
x_train = x_train.astype('float32')/255
y_train = y_train.astype('float32')
x_test = x_test.astype('float32')/255

# Converting data to numpy array

In [ ]:
x_train= x_train.to_numpy()
y_train= y_train.to_numpy()
x_test = x_test.to_numpy()

# Reshaping the array

In [ ]:
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

# Converting Y train into different categories

In [ ]:
y_train = keras.utils.to_categorical(y_train, 10)

# Model building

In [ ]:
""""model = keras.Sequential()
model.add(keras.layers.Conv2D(32,3,activation="relu",padding = 'same',input_shape=(28,28,1)))
#model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Conv2D(32,3,activation="relu",padding = 'same'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))"""
vgg_model = keras.Sequential()
from tensorflow.keras.layers import Dense, MaxPool2D, Flatten, Conv2D, Dropout
vgg_model.add(Conv2D(32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape=(28,28,1)))
vgg_model.add(Conv2D(32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
vgg_model.add(MaxPool2D(pool_size=(2,2)))
vgg_model.add(Dropout(0.5*0.5))

vgg_model.add(Conv2D(64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
vgg_model.add(Conv2D(64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
vgg_model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
vgg_model.add(Dropout(0.5*0.5))

vgg_model.add(Flatten())
vgg_model.add(Dense(256, activation = "relu"))
vgg_model.add(Dropout(0.5))
vgg_model.add(Dense(10, activation = "softmax"))

In [ ]:
vgg_model.compile(optimizer="sgd",loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
vgg_model.summary()

In [ ]:
history = vgg_model.fit(x_train,y_train,epochs=21)

In [ ]:
df1 = pd.DataFrame(history.history)

In [ ]:
df1["accuracy"].plot()

In [ ]:
df1["loss"].plot()

In [ ]:
preds = vgg_model.predict(x_test)

In [ ]:
results = np.argmax(preds,axis=1)
results = pd.Series(results, name="Label")
list=[]
[list.append(i) for i in range(1,28001)]
ImageID = pd.Series(list, name="ImageID").astype("int32")
submission = pd.concat([ImageID,results],axis = 1)
submission.to_csv("preds.csv", index=False)
